### python的函数都是一等函数
一等函数指的是：
- 在运行时创建
- 能赋值给变量或数据结构中的元素
- 能作为参数传给函数
- 能作为函数的返回结果

要把函数视作对象

In [ ]:
def factorial(n):
  '''
  return n!
  '''
  
  return n if n <2 else n*factorial(n-1)

print(factorial(5))
print(type(factorial)) # <class 'function'>

fac = factorial
print(fac(6))
print(map(fac,range(6)))
print(list(map(fac,range(10))))

#### 高阶函数
能够接受函数为参数或者把函数作为结果返回的函数都是高阶函数，比如map，reduce，filter

In [ ]:
fruits = ['strawberry','banana','apple','cherry','raspberry']
print(sorted(fruits,key=len))

def reverse(word):
  return word[::-1] # 调换字母顺序

print(sorted(fruits,key = reverse))

用列表推倒式取代部分高阶函数：
- list(map(fact,range(10)))，可以用[fact(n) for n in range(10)]
- list(map(fact, filter(lambda n:n%2,range(10)))),可以用[fact(n) for n in range(10) if n % 2]

In [ ]:
# 少用匿名函数，难读且有语法限制，
fruits = ['banana','apple','cherry']
print(sorted(fruits,key = lambda fruit:fruit[::-1]))

#### 可调用对象
可调用对象即为可以被调用运算符（即（））应用的对象，callable（）返回true的对象。有以下7种：
- def或lambda创建的函数
- 内置函数，如len，time.strftime
- 内置方法，如dict.get，dict.setdefault
- 类，因为类新建时候是调用__new__和__init__的
- 类的实例，如果类定义了__call__方法，那么它的实例可以作为函数调用
- 生成器函数，yield函数返回生成器对象

In [ ]:
# 实现__call__，得到一个可以当方法调用的类实例
import random

class BingoCage:
  def __init__(self,items):
    self._items = list(items)
    random.shuffle(self._items)
  
  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty bingocage')
  
  def __call__(self):
    return self.pick()
  
bingo = BingoCage(range(3))

print(bingo.pick())
print(bingo()) #实例当方法用

In [ ]:
# 一些函数有而普通对象没有的属性
class C:
  pass

def f():
  pass

c = C()
print(sorted(set(dir(f))-set(dir(c))))

In [ ]:
# 定位参数和仅限关键字参数
def tag(name, *content, cls=None, **attrs):
    if cls is not None:
        attrs['class'] = cls
    if attrs:
        attrs_str = ''.join(f' {attr}="{val}"' for attr, val in sorted(attrs.items()))
    else:
        attrs_str = ''
    if content:
        return '\n'.join(f'<{name}{attrs_str}>{c}</{name}>' for c in content)
    else:
        return f'<{name}{attrs_str}/>'

print(tag('br'))
print(tag('p','hello'))
print(tag('p','hello','world',cls='sidebar'))
print(tag('p','hello','world',cls='sidebar',align='center'))


#### 参数的内省
函数作为一个对象，他的参数实际上也保存在函数对象中。
- \_\_defaults\_\_属性的值是一个元组，保存了定位参数和关键字参数的默认值
- \_\_kwdefaults\_\_属性保存了仅限关键字参数的默认值
- \_\_code\_\_保存了参数的名称

In [ ]:
def clip(text, max_len=80):
  end = None
  if len(text) > max_len:
    space_before = text.rfind(' ',0,max_len)
    if space_before >= 0 :
      end = space_before
    else:
      space_after = text.rfind(' ',max_len)
      if space_after >= 0:
        end = space_after
  if end is None:
    end = len(text)
  return text[:end].rstrip()

print(clip.__defaults__)
print(clip.__code__.co_varnames)

from inspect import signature

sig = signature(clip)
print(sig)
for name ,param in sig.parameters.items():
  print(param.kind,':',name,'=',param.default)

#### 函数注解（annotation或typehints）
annotation和typehints是同一个东西，可以为函数参数和返回值附加元数据，会存储到函数的__annotations__属性中，仅此而已。注解只是元数据，可以给IDE，框架，装饰器等工具使用。

In [ ]:
def clip(text:str, max_len:'int > 0'=80) -> str:
  end = None
  if len(text) > max_len:
    space_before = text.rfind(' ',0,max_len)
    if space_before >= 0 :
      end = space_before
    else:
      space_after = text.rfind(' ',max_len)
      if space_after >= 0:
        end = space_after
  if end is None:
    end = len(text)
  return text[:end].rstrip()

print(clip.__annotations__)

from inspect import signature

sig = signature(clip)
for param in sig.parameters.values():
  print(param.annotation,param.name,param.default)

#### 函数式编程
可以使用operator和functools这2个包来做函数式编程

In [ ]:
from functools import reduce
def fact(n):
  return reduce(lambda a,b:a*b,range(1,n+1))

from operator import mul

def fact2(n):
  return reduce(mul,range(1,n+1))

In [ ]:
# 使用itemgetter
from operator import itemgetter

data = [('Tokyo','JP'),
        ('New York','US'),
        ('Mexico city','MX')
      ]
getter = itemgetter(1)

for item in sorted(data,key=getter):
  print(item)
  
for city in data:
  print(itemgetter(1,0)(city))

In [ ]:
# partial冻结参数
from operator import mul
from functools import partial

# 冻结mul的第一个参数为3
triple = partial(mul,3)
triple(7)

tuple(map(triple,range(1,10)))